## Langgraph

In this notebook, we will learn the basics of langgraph and how to create a basic workflow using langgraph. We will also create one mini project with crewAI and Langgraph to understand it in more detail.

First of all let's create a basic workflow using langgraph. So let's start by installing the required dependencies


In [ ]:
!pip install langgraph langchain openai langchain_openai langchain_community pyowm

We will use open weather API to get information about weather information of any location. You can get your own API key from their website for free. Additionally, you will also require OpenAI API key to use their LLM.

Store your openweather and OpenAI API keys in environment variables

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openai_secret
os.environ["OPENWEATHERMAP_API_KEY"] = openweather_secret

We will use “OpenWeatherMapAPIWrapper()” to make a call to OpenWeather API and we will use “ChatOpenAI” to use GPT-3.5 model.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import OpenWeatherMapAPIWrapper
openai_llm = ChatOpenAI(temperature=0.4)
weather = OpenWeatherMapAPIWrapper()

We will create one node which will take user input and extract the city name from user’s query and pass it to next node where we will get the weather information using openweather wrapper

In [ ]:
# Node to extract city from user input
def agent(input_1):
  res = openai_llm.invoke(f"""
  You are given one question and you have to extract city name from it
  Don't respond anything except the city name and don't reply anything if you can't find city name

  Here is the question:
  {input_1}
  """)
  return res.content
  
# Node to find weather information
def weather_tool(input_2):
  data = weather.run(input_2)
  return data

Now let’s connect these 2 nodes using edges and create a graph

In [ ]:
from langgraph.graph import Graph
workflow = Graph()

workflow.add_node("agent", agent)
workflow.add_node("weather", weather_tool)

# Connecting 2 nodes
workflow.add_edge('agent', 'weather')

Here is how our workflow will look like 👇
![](https://assets-global.website-files.com/62528d398a42420e66390ef9/6641f6f8558143699a109f3b_image3.png)

Additionally, you can define the starting and ending points of your workflow. Here we know that our input will be passed to an agent and then we will find the weather info so the starting point will be the agent node and the ending point will be the weather node.

In [ ]:
workflow.set_entry_point("agent")
workflow.set_finish_point("weather")

app = workflow.compile()

And finally, let’s run our langgraph!

In [ ]:
app.invoke("What is weather in delhi?")

And you will get output like this

```
In Delhi, the current weather is as follows:
Detailed status: haze
Wind speed: 3.6 m/s, direction: 80°
Humidity: 34%
Temperature: 
  - Current: 37.05°C
  - High: 37.05°C
  - Low: 37.05°C
  - Feels like: 39.1°C
Rain: {}
Heat index: None
Cloud cover: 0%
```

But the output is not properly readable, what if we have a responder node that will format this output properly and work as a weather agent instead of a function.

Let’s add a new node called ‘responder’ which will format the weather tool output and provide better results. But wait 🤔, we will need the user’s query to properly format our answer but we will only get the weather tool output in our node so how can we get the user’s query which we are passing to agent node? 👀

This is where states comes into picture. We will create one state called “messages” which will store all the conversation happening in the entire workflow. so let’s create it first!

In [ ]:
# We will keep adding our conversation in this list
from typing import TypedDict, Annotated, Sequence
import operator
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]

And now let’s create our third node

In [ ]:
def responder(state):
  agent = openai_llm.invoke(f"""
  You have given a weather information and you have to respond to user's query based on the information

  Here is the user query:
  ---
  {state["messages"][0]}
  ---

  Here is the information:
  ---
  {state["messages"][2]}
  ---
  """)
  return {"messages": [agent.content]}

Make sure first 2 nodes are also using state and adding their responses in messages

In [ ]:
def agent(state):
  query = state["messages"]
  res = openai_llm.invoke(f"""
  You are given one question and you have to extract city name from it

  Only reply the city name if it exists or reply 'no_response' if there is no city name in question

  Here is the question:
  {query[0]}
  """)
  return {"messages":[res.content]}

def weather_tool(state):
  context = state["messages"]
  city_name = context[1]
  data = weather.run(city_name)
  return {"messages": [data]}

Finally let’s create a stateful graph as we have state to pass between nodes. Also define nodes and connect them using edges.

In [ ]:
from langgraph.graph import StateGraph
workflow = StateGraph(AgentState)
workflow.add_node('agent',agent)
workflow.add_node('weather',weather_tool)
workflow.add_node("responder",responder)

# Connect the nodes
workflow.add_edge('agent', 'weather')
workflow.add_edge('weather', 'responder')

# Set entry and finish point
workflow.set_entry_point("agent")
workflow.set_finish_point("responder")
app = workflow.compile()

This is how the new workflow will look like 👇
![](https://assets-global.website-files.com/62528d398a42420e66390ef9/6641f7d993f204bc6be3def1_image4.png)

Let’s try it with responder!

In [ ]:
inputs = {"messages": ["What is weather in delhi?"]}
response = app.invoke(inputs)
print(response['messages'][-1])

And now the agent will reply in more readable way

`In Delhi, the current weather is hazy with a temperature of 37.05°C. The wind speed is 3.6 m/s coming from the direction of 80°. The humidity is at 34% and there is no rain expected. The cloud cover is at 0% and it currently feels like 39.1°C.`

You can also ask specific questions like “Is there any chance of rain in delhi?” or “Is it a best weather to go on long drive in delhi?” and responder will respond accordingly.

Still there is a one problem 💀, the agent still can’t answer questions like “How are you?” because it will try to find a city name in it and if it can’t find it then weather tool will throw an error so we will have to handle this case too.

![](https://assets-global.website-files.com/62528d398a42420e66390ef9/6641f828beb30e36fa4deba1_image5.png)

What if we run weather tool conditionally? 🤔, means we will only run that tool when user asks weather information otherwise we will not respond anything. This is where we will need to create a conditional edge so let’s create one!

We will tell our llm to response with “no_response” if it can’t find city name in user query and based on that output, we will use weather tool or end the workflow.

This is how the updated workflow will look like 👇

![](https://assets-global.website-files.com/62528d398a42420e66390ef9/6641f8542404f91709fe6519_image6.png)

In [ ]:
from langgraph.graph import Graph, END
# Defining condition function
def where_to_go(state):
  ctx = messages[0]
  if ctx == "no_response":
    return "end"
  else:
    return "continue"
  
# Create an conditional edge
workflow.add_conditional_edges('agent',where_to_go,{
    "end": END,
    "continue": "weather_tool"
})
# Remaining edges
workflow.add_edge("weather_tool","responder")

Now if we ask questions like “How are you?” then it will respond with “no_response” instead of throwing an error and if you ask questions regarding weather information then it will use weather tool and respond with responder 🚀!

Now we know how to create a basic workflow using langgraph so let’s get our hand dirty by creating a multi purpose AI agent👀!

## Mini Project: Let’s Create a Multi-Purpose AI Agent
Let’s create an AI agent that can give us live weather information, create draft replies for our emails or even it can chat with us normally like a chatbot. You can add more workflows in this agent but for the simplicity of this blog I am only going to add 2 workflows. So let’s get started 🚀!

## Workflow
Before creating the project, let’s take a look at the workflow of our agent!
![](https://assets-global.website-files.com/62528d398a42420e66390ef9/6641f8896dd1ef4f1d90e806_image7.png)

We will first add the user input in our entry node where the user input will be categorized into 3 categories:

- email_query: If user want to generate an email response to given email
- weather_query: If user want weather information about any location
- other: If user want any other information

Now based on the categories, we will redirect the query to right node. 🔂

We will use CrewAI to create a crew which can categorize the email and then based on the category it will write a response. We will also create an separate agent for weather where we will provide the openweather function as a tool and it will automatically format the final weather information response. For all other queries, we will just make a simple OpenAI call.

### Prerequisites
Here are the things you will need to create this project

- OpenAI API key
- Openweather API key
- A basic knowledge about CrewAI (If you don’t know about CrewAI then I suggest you to take a look at my blog about CrewAI)

Add your API keys in environment variables

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openai_secret
os.environ["OPENWEATHERMAP_API_KEY"] = openweather_secret

So now we have everything ready, let’s get back to coding 💻!

### Let’s code it

First of all, let’s install required dependencies!

In [ ]:
!pip install langgraph langchain openai langchain_openai langchain_community pyowm crewai

Let’s initialize ChatOpenAI and OpenweatherWrapper objects

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import OpenWeatherMapAPIWrapper
openai_llm = ChatOpenAI(temperature=0.4)
weather = OpenWeatherMapAPIWrapper()

Let’s import required packages and modules

In [ ]:
# Import required dependencies
from crewai import Crew, Agent, Task
from textwrap import dedent
import os
import json
import requests

Before creating agents or workflows, let’s first define the states which we are going to pass among nodes.

We will use these state variables in our workflow 👇

- messages: It will store the conversation history to keep track of conversation happening in workflow between agents
- email: If user want to generate a email response then entry node will extract the email body from user input and add it in this state
- query: It will store the user’s query
- category: It will store the category of user’s query ( email_query, weather_query or other )

Let’s write the code to define our states:

In [ ]:
from typing import TypedDict
class AgentState(TypedDict):
    messages: list[str]
    email: str
    query: str
    category: str

First let’s create our email crew which will have 2 agents: ClassifierAgent and emailWriterAgent

ClassifierAgent will classify the given email in Important or Casual Category and emailWriterAgent will generate response based on the category.

In [ ]:
class Agents:
	def classifierAgent():
	    return Agent(
	      role='Email Classifier',
	      goal='You will be given an email and you have to classify the given email in one of these 2 categories: 1) Important 2) Casual ',
	      backstory='An email classifier who is expert in classifying every type of email and have classified so many emails so far',
	      verbose = True,
	      allow_delegation=False,
	    )
	def emailWriterAgent():
	  return Agent(
	    role='Email writing expert',
	    goal="You are email writing assistant for Shivam. You will be given an email and a category of that email and your job is to write a reply for that email. If email category is 'Important' then write the reply in professional way and If email category is 'Casual' then write in a casual way",
	    backstory='An email writer with an expertise in email writing for more than 10 years',
	    verbose = True,
	    allow_delegation=False,
	  )

Now let’s create the tasks for these agents

In [ ]:
class Tasks:
	def classificationTask(agent,email):
	    return Task(
	        description=dedent(f"""
	        You have given an email and you have to classify this email
	        {email}
	        """),
	        agent = agent,
	        expected_output = "Email category as a string"
	    )
	def writerTask(agent,email):
	  return Task(
	      description=dedent(f"""
	      Create an email response to the given email based on the category provided by 'Email Classifier' Agent
	      {email}
	      """),
	      agent = agent,
	      expected_output = "A very concise response to the email based on the category provided by 'Email Classifier' Agent"
	  )

Finally let’s create our email crew

In [ ]:
class EmailCrew:
  def __init__(self,email):
    self.email = email
  def run(self):
		# Agents
    classifierAgent = Agents.classifierAgent()
    writerAgent = Agents.emailWriterAgent()
		# Tasks
    classifierTask = Tasks.classificationTask(agent=classifierAgent,email=self.email)
    writerTask = Tasks.writerTask(agent=writerAgent,email=self.email)
		# Create crew
    crew = Crew(
      agents=[classifierAgent,writerAgent],
      tasks=[classifierTask,writerTask],
      verbose=2, # You can set it to 1 or 2 to different logging levels
    )
		# Run the crew
    result = crew.kickoff()
    return result

Finally, let’s create a node where we will run this crew

In [ ]:
class Nodes:
  def writerNode(self,state):
    email = state["email"]
    emailCrew = EmailCrew(email)
    crewResult = emailCrew.run()
    messages = state["messages"]
    messages.append(crewResult)
    return {"messages": messages}

Now let’s create our weather agent workflow 🌤️

We will create a weather tool which will use openweather wrapper to get the weather information. We will assign this tool to our weather agent.

In [ ]:
from langchain.tools import tool
class Tools:
  @tool("Tool to get the weather of any location")
  def weather_tool(location):
    """
    Use this tool when you have given a location and you want to find the weather of that location
    """
    data = weather.run(location)
    return data
    
class Agents:
	# ... Other agents
  def weatherAgent():
    return Agent(
          role = 'Weather Expert',
          goal = 'You will be given a location name and you have to find the weather information about that location using the tools provided to you',
          backstory = "An weather expert who is expert in providing weather information about any location",
          tools = [Tools.weather_tool],
          verbose = True,
          allow_delegation = False,
      )

Let’s create a task for our weather agent

In [ ]:
class Tasks:
	# ... Other tasks
    def weatherTask(agent,query):
        return Task(
            description = dedent(f"""
            Get the location from the user query and find the weather information about that location

            Here is the user query:
            {query}
            """),
            agent = agent,
            expected_output = "A weather information asked by user"
        )

Finally, create a node where we will run this agent:

In [ ]:
class Nodes:
	# ... Other nodes
    def weatherNode(self,state):
        query = state["query"]
        weatherAgent = Agents.weatherAgent()
        weatherTask = Tasks.weatherTask(agent=weatherAgent,query=query)
        result = weatherTask.execute()
        messages = state["messages"]
        messages.append(result)
        return {"messages": messages}

Let’s create entry and reply node as well

In [ ]:
class Nodes:
	# ... Other nodes
    def replyNode(self,state):
        query = state["query"]
        agent = openai_llm.invoke(f"""
            {query}
        """)
        messages = state["messages"]
        messages.append(agent.content)
        return {"messages": messages}
    def entryNode(self,state):
        input = state["query"]
        agent = openai_llm.invoke(f"""
        User input
        ---
        {input}
        ---
        You have given one user input and you have to perform actions on it based on given instructions

        Categorize the user input in below categories
        email_query: If user want to generate a reply to given email
        weather_query: If user want any weather info about given location
        other: If it is any other query

        After categorizing your final RESPONSE must be in json format with these properties:
        category: category of user input
        email: If category is 'email_query' then extract the email body from user input with proper line breaks and add it here else keep it blank
        query: If category is 'weather_query' or 'other' then add the user's query here else keep it blank
        """)
        response = json.loads(agent.content)
        return {'email': response["email"], 'query': response['query'], 'category': response['category']}

And we have successfully created all the nodes for our workflow!

Now let’s define the condition function which will decide the conditional flow for our conditional edge based on the category

In [ ]:
def where_to_go(state):
  cat = state['category']
  print("Category: ",cat)
  if cat == "email_query":
    return "email"
  elif cat == "weather_query":
    return "weather"
  else:
    return "reply"

And finally let’s create the stateful graph and define our nodes in it

In [ ]:
from langgraph.graph import Graph, END, StateGraph
workflow = StateGraph(AgentState)
node = Nodes()
workflow.add_node('entryNode',node.entryNode)
workflow.add_node('weatherNode',node.weatherNode)
workflow.add_node("responder",node.replyNode)
workflow.add_node('emailNode',node.writerNode)

Let’s connect these nodes using edges

In [ ]:
workflow.add_conditional_edges('entryNode',where_to_go,{
    "email": "emailNode",
    "weather": "weatherNode",
    "reply": "responder"
})
workflow.add_edge("weatherNode",END)
workflow.add_edge("responder",END)
workflow.add_edge("emailNode",END)

workflow.set_entry_point("entryNode")
app = workflow.compile()

And now it’s time to test our agent 👀!

In [ ]:
query = """
Can you reply to this email

Hello,
Thank you for applying to xyz company
can you share me your previous CTC
Thanks,
HR
"""
inputs = {"query": query, "messages": [query]}
result = app.invoke(inputs)
print("Agent Response:",result['messages'][-1])

After running the above code, you will see the query got categorized as “email_query” and then using EmailCrew it will generate the reply for the extracted email which looks like this:

```
Agent Says: Subject: Re: Application to XYZ Company

Dear [HR's Name],

Thank you for considering my application for the position at XYZ Company.

As per your request, my previous CTC was [mention CTC]. I am open to any negotiation based on the job requirements and the value I can bring to your team at XYZ Company.

Thank you once again for the opportunity. I look forward to potentially furthering the application process.

Best Regards,
[Your Name]
```

Ofc you can make it better with better prompts but I will leave it on you so that you can do experiment with it.

You can also try with below queries for different use cases and agent will reply differently.

weather_query: 
```
Is there any chance of rain in delhi today?
```

email_query:
```
Hey can you generate a reply to this email

Hey man,
I just saw your portfolio and quite liked it. Can you tell me which languages do you use the most
Thanks
```

other:
```
Hello how are you?
```